# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [2]:
env = UnityEnvironment(file_name='./Reacher.app')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [5]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

Total score (averaged over agents) this episode: 0.17049999618902803


When finished, you can close the environment.

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

# Project 2: Continuous Control

### External Dependencies

In [1]:
import numpy as np
import logging
import os
import pandas as pd
from pathlib import Path
import torch
import torch.nn.functional as F

seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

### Internal Dependencies

In [2]:
from unityagents import UnityEnvironment
from rl_library.agents.ddpg_agent import DDPGAgent
from rl_library.agents.models.bodies import SimpleNeuralNetBody
from rl_library.agents.models.heads import SimpleNeuralNetHead, DeepNeuralNetHeadCritic
from rl_library.monitors.unity_monitor import UnityMonitor

### Logger

In [3]:
save_path = f"./results/MultiAgents_DDPG_{pd.Timestamp.utcnow().value}"
os.makedirs(save_path, exist_ok=True)

logger = logging.getLogger()
logger.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s : %(message)s')

handler = logging.FileHandler(f"{save_path}/logs_navigation_{pd.Timestamp.utcnow().value}.log")
handler.setLevel(logging.DEBUG)
handler.setFormatter(formatter)
logger.addHandler(handler)

### Configuration

In [4]:
n_episodes = 250
config = dict(
    # Environment parameters
    env_name="Reacher",
    n_episodes=n_episodes,
    length_episode=1500,
    save_every=100,
    save_path=save_path,
    mode="train",  # "train" or "test"
    evaluate_every=5000,  # Number of training episodes before 1 evaluation episode
    eps_decay=1,  # Epsilon decay rate

    # Agent Parameters
    agent="DDPG",
    hidden_layers_actor=(200, 150),  # (50, 50, 15),  # (200, 150),  #
    hidden_layers_critic_body=(400,),  # (50, 50,),  #
    hidden_layers_critic_head=(300,),  # (50,),   # (300,)
    func_critic_body="F.leaky_relu",  #
    func_critic_head="F.leaky_relu",  #
    func_actor_body="F.leaky_relu",  #
    lr_scheduler=None,

    # Hyper parameters
    TAU=1e-3,  # for soft update of target parameters
    BUFFER_SIZE=int(1e6),  # replay buffer size
    BATCH_SIZE=128,  # minibatch size
    GAMMA=0.99,  # discount factor
    LR_ACTOR=1e-3,  # learning rate of the actor
    LR_CRITIC=1e-3,  # learning rate of the critic
    WEIGHT_DECAY=0,  # L2 weight decay
    UPDATE_EVERY=1,  # Number of actions before making a learning step
    
    action_noise="OU",  #
    action_noise_scale=1,
    weights_noise=None,  #
    state_normalizer=None,  # "RunningMeanStd"

    warmup=0,  # Number of random actions to start with as a warm-up
    start_time=str(pd.Timestamp.utcnow()),
    random_seed=seed,
    threshold=30
)

## 1. Initialization

### 1. Start the Environment

In [5]:
env = UnityEnvironment(file_name=f'./{config["env_name"]}')  # mac OS

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [6]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)
config["n_agents"] = num_agents

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])
config.update(dict(action_size=action_size, state_size=state_size))

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


##  2. Training

### Unity Monitor

In [7]:
monitor = UnityMonitor(env=env, config=config)

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  7.90150833e+00 -1.00000000e+00
  1.25147629e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -5.22214413e-01]


### Actor and Critic models

In [8]:
# Actor model
seed = 0
actor = SimpleNeuralNetHead(action_size,
                            SimpleNeuralNetBody(state_size, config["hidden_layers_actor"], seed=seed),
                            func=F.tanh, seed=seed)
actor_target = SimpleNeuralNetHead(action_size,
                                   SimpleNeuralNetBody(state_size, config["hidden_layers_actor"], seed=seed),
                                   func=F.tanh, seed=seed)
# Critic model
critic = DeepNeuralNetHeadCritic(action_size,
                                 SimpleNeuralNetBody(state_size, config["hidden_layers_critic_body"],
                                                     func=eval(config["func_critic_body"]), seed=seed),
                                 hidden_layers_sizes=config["hidden_layers_critic_head"],
                                 func=eval(config["func_critic_head"]),
                                 end_func=None, seed=seed)

critic_target = DeepNeuralNetHeadCritic(action_size,
                                        SimpleNeuralNetBody(state_size, config["hidden_layers_critic_body"],
                                                            func=eval(config["func_critic_body"]), seed=seed),
                                        hidden_layers_sizes=config["hidden_layers_critic_head"],
                                        func=eval(config["func_critic_head"]),
                                        end_func=None, seed=seed)

INFO:rllib.models:Initialized SimpleNeuralNetHead with body : ModuleList(
  (0): Linear(in_features=33, out_features=200, bias=True)
  (1): Linear(in_features=200, out_features=150, bias=True)
) and head Linear(in_features=150, out_features=4, bias=True)
INFO:rllib.models:state_dict= OrderedDict([('body.layers.0.weight', tensor([[ 0.0069,  0.0236,  0.0656,  ...,  0.0022,  0.0687,  0.0359],
        [ 0.0332, -0.0127, -0.0178,  ...,  0.0119,  0.0394,  0.0308],
        [ 0.0394, -0.0615,  0.0259,  ..., -0.0399,  0.0312,  0.0176],
        ...,
        [ 0.0686, -0.0601, -0.0615,  ..., -0.0107,  0.0513, -0.0177],
        [-0.0322, -0.0090,  0.0421,  ..., -0.0328,  0.0604,  0.0343],
        [-0.0611,  0.0538,  0.0468,  ...,  0.0646, -0.0550,  0.0647]])), ('body.layers.0.bias', tensor([ 0.1632, -0.0778, -0.0771, -0.0725, -0.1326,  0.1624, -0.0983, -0.0783,
        -0.0527,  0.1465,  0.0812,  0.0641, -0.0458,  0.0880,  0.0316,  0.1387,
         0.0386,  0.1278,  0.1201,  0.0304, -0.1668,  0.03

INFO:rllib.models:Initialized SimpleNeuralNetHead with body : ModuleList(
  (0): Linear(in_features=33, out_features=200, bias=True)
  (1): Linear(in_features=200, out_features=150, bias=True)
) and head Linear(in_features=150, out_features=4, bias=True)
INFO:rllib.models:state_dict= OrderedDict([('body.layers.0.weight', tensor([[-0.0289,  0.0285, -0.0238,  ...,  0.0634, -0.0549,  0.0473],
        [-0.0586, -0.0493,  0.0679,  ...,  0.0014, -0.0119, -0.0104],
        [-0.0392, -0.0300,  0.0552,  ..., -0.0311, -0.0150, -0.0115],
        ...,
        [ 0.0248, -0.0296,  0.0376,  ..., -0.0469, -0.0043,  0.0480],
        [ 0.0182,  0.0685, -0.0453,  ..., -0.0488,  0.0606, -0.0243],
        [ 0.0401, -0.0551, -0.0440,  ...,  0.0312, -0.0467,  0.0116]])), ('body.layers.0.bias', tensor([ 0.0682,  0.0751,  0.1120,  0.0074, -0.1022,  0.0554,  0.0007, -0.1266,
        -0.0342, -0.1410, -0.0395,  0.1268,  0.0886,  0.0249, -0.0366, -0.0667,
        -0.1547,  0.1382,  0.1331,  0.0238, -0.1414,  0.04

INFO:rllib.models:Initialized DeepNeuralNetHeadCritic with body : ModuleList(
  (0): Linear(in_features=33, out_features=400, bias=True)
) and head ModuleList(
  (0): Linear(in_features=404, out_features=300, bias=True)
  (1): Linear(in_features=300, out_features=1, bias=True)
)
INFO:rllib.models:state_dict= OrderedDict([('body.layers.0.weight', tensor([[-1.0039e-03, -2.7219e-02, -5.8732e-03,  ..., -3.5667e-02,
          1.3713e-02,  4.2372e-02],
        [ 3.1677e-02,  2.4817e-02, -4.7929e-02,  ...,  4.7579e-02,
         -1.8188e-02, -1.3225e-02],
        [ 2.7205e-03, -2.6465e-02,  2.5049e-02,  ..., -1.4964e-02,
          5.4236e-03,  3.4415e-02],
        ...,
        [ 1.4893e-02,  4.9692e-02,  4.9638e-02,  ..., -3.5418e-03,
         -7.0493e-03, -2.8474e-02],
        [ 1.9417e-02,  6.5405e-05, -2.5932e-03,  ..., -4.7790e-02,
          3.9044e-02,  3.7796e-03],
        [-1.8730e-02, -2.0211e-02, -3.4439e-02,  ..., -2.7822e-02,
          1.8316e-03, -4.1569e-02]])), ('body.layers.0.bi

INFO:rllib.models:Initialized DeepNeuralNetHeadCritic with body : ModuleList(
  (0): Linear(in_features=33, out_features=400, bias=True)
) and head ModuleList(
  (0): Linear(in_features=404, out_features=300, bias=True)
  (1): Linear(in_features=300, out_features=1, bias=True)
)
INFO:rllib.models:state_dict= OrderedDict([('body.layers.0.weight', tensor([[-0.0415,  0.0021, -0.0361,  ...,  0.0424, -0.0358, -0.0475],
        [ 0.0418,  0.0027,  0.0028,  ..., -0.0166,  0.0357,  0.0261],
        [-0.0232, -0.0133, -0.0087,  ...,  0.0396,  0.0155,  0.0406],
        ...,
        [-0.0305, -0.0472, -0.0194,  ..., -0.0244,  0.0007, -0.0407],
        [ 0.0030, -0.0292,  0.0170,  ...,  0.0320, -0.0019,  0.0346],
        [ 0.0211, -0.0077, -0.0079,  ...,  0.0385, -0.0280, -0.0004]])), ('body.layers.0.bias', tensor([-0.1432,  0.1058,  0.1576, -0.1404,  0.0560, -0.1501, -0.0688,  0.1134,
        -0.0649,  0.0991, -0.0447,  0.1599,  0.0862,  0.0472,  0.1621,  0.1170,
         0.0798,  0.0187,  0.0888

### DDPG Agent

In [9]:
agent = DDPGAgent(state_size=state_size, action_size=action_size,
                  model_actor=actor, model_critic=critic,
                  actor_target=actor_target, critic_target=critic_target,
                  action_space_low=-1, action_space_high=1,
                  config=config,
                  )
agent.debug_mode = False  # Current default debug_mode for agent is True, which would print out most variables 
# every 10000 steps of the agent. Turning off to not overwhelm the jupyter notebook.

INFO:rllib.ddpgagent:Actor Optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)
INFO:rllib.ddpgagent:Critic Optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)
INFO:rllib.ddpgagent:Actor LR Scheduler: None
INFO:rllib.ddpgagent:Critic LR Scheduler: None
INFO:rllib.ddpgagent:Initiated state_normalizer=None, reward_normalizer=None


### Training

In [ ]:
start = pd.Timestamp.utcnow()
scores = monitor.run(agent)
logger.info("Average Score last 100 episodes: {}".format(np.mean(scores[-100:])))
elapsed_time = pd.Timedelta(pd.Timestamp.utcnow() - start).total_seconds()
logger.info(f"Elapsed Time: {elapsed_time} seconds")

/Users/quentincangelosi/anaconda3/envs/rl/lib/python3.6/site-packages/torch/nn/functional.py:1614: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
INFO:rllib.monitor:Episode 1    Average Score: 0.73, Agent Loss: [3.34303601e-02 4.31243749e-05], Last Score: avg=0.73, min=0.23, max=1.31 (1000 steps), eps: 1.00
INFO:rllib.monitor:Episode 2    Average Score: 0.66, Agent Loss: [2.24504207e-02 2.91313480e-05], Last Score: avg=0.58, min=0.00, max=1.41 (1000 steps), eps: 1.00
INFO:rllib.monitor:Episode 3    Average Score: 0.65, Agent Loss: [1.72723618e-02 2.50832856e-05], Last Score: avg=0.64, min=0.00, max=1.56 (1000 steps), eps: 1.00
INFO:rllib.monitor:Episode 4    Average Score: 0.70, Agent Loss: [1.37112641e-02 2.47906114e-05], Last Score: avg=0.84, min=0.19, max=1.74 (1000 steps), eps: 1.00
INFO:rllib.monitor:Episode 5    Average Score: 0.73, Agent Loss: [1.07426337e-02 2.57104234e-05], Las

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sb
sb.set()
sb.set_style("darkgrid", {"axes.facecolor": ".9"})
sb.despine()
sb.set_context("talk")

In [ ]:
%matplotlib inline

df = pd.DataFrame({"scores": np.mean(scores, axis=1)})
df["episode"] = df.index + 1
df["rolling_score_100"] = df["scores"].rolling(100).mean()

plt.figure(figsize=(15, 8))
sb.lineplot(x="episode", y="scores", data=df)
sb.lineplot(x="episode", y="rolling_score_100", data=df)
plt.axhline(30, c="r", ls="--", label="Objective: 30")
plt.legend()
plt.show()

## 3. Testing

In [ ]:
monitor.mode = "test"
monitor.n_episodes = 10

agent = DDPGAgent.load(filepath=config['save_path'], mode="test")

scores = monitor.run(agent)

logger.info(f"Test Score over {len(scores)} episodes: {np.mean(scores)}")

#### When finished, you can close the environment.

In [ ]:
logger.info("Closing...")
env.close()